<a href="https://colab.research.google.com/github/joyuno/lguplus_project/blob/main/%EC%BB%A4%EB%AE%A4%EB%8B%88%ED%8B%B0_%EC%A1%B0%EC%82%AC_%EC%99%84%EC%84%B1%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install selenium
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def selenium_scroll_option():
  SCROLL_PAUSE_SEC = 3

  last_height = driver.execute_script("return document.body.scrollHeight")

  while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(SCROLL_PAUSE_SEC)

    new_height = driver.execute_script("return document.body.scrollHeight")
#revolution_main_table > tbody > tr:nth-child(3) > td:nth-child(2) > a > span
#revolution_main_table > tbody > tr:nth-child(4) > td:nth-child(2) > a > span

    if new_height == last_height:
        break
    last_height = new_height

In [ ]:
import selenium
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import re
import sys
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=chrome_options)
# 키워드 목록
keywords = ["이야기","마블","헬모","아이즈","유모바일","모빙","티플러스","알닷","에르엘","프리티","슈가","Sugar","sugar",
            "에이모바일","kg","에이모", "찬스","핀다이렉트", "이지모바일","스노우맨", "스노우","보스","프리텔","와이엘","와이엘랜드",
            "YL","A모바일", "벨류컴","인스모바일","인스","리브엠","리브모바일","모나","토스모바일","토스","유플","코나아이","EG",
            "에넥스","헬로","스마텔","위너스텔","친구모바일","에넥스","유니컴즈","KPMO","KB","에스원","엔티온","LG","lg",
            "헬로비전","핀샷","화인","CK","ck","에이프러스","슈거","유모알","유모비","liv","핀다","시월"]

# 크롤링할 페이지 범위
start_page = 1
end_page = 2

# 두 글자 이상 키워드 패턴 생성
keyword_pattern = re.compile('|'.join(re.escape(keyword) for keyword in keywords if len(keyword) >= 2))

base_url = 'https://www.ppomppu.co.kr/zboard/zboard.php?id=phone&page={page_num}&category=6&divpage=691'
title = []
titles = []
paragraphs = []
for page_num in range(start_page, end_page + 1):
    url = base_url.format(page_num=page_num)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    table = soup.find(id="revolution_main_table")
    rows = table.find_all("tr")

    for row in rows:
        cells = row.find_all("td", class_="baseList-space")

        for cell in cells:
            a_tag = cell.find("a", class_="baseList-title")

            if a_tag:
                spans = a_tag.find_all("span")  # select 대신 find_all 사용하여 모든 span 요소 가져옴
                for span in spans:
                    span_text = span.get_text().strip()
                    # 두 글자 이상 일치하는 키워드가 있는 경우
                    if keyword_pattern.search(span_text):
                        #title = [text for text in span_text if keyword_pattern.search(text)]
                        title.append(span_text)
                        href = a_tag['href']
                        full_link = "https://www.ppomppu.co.kr/zboard/" + href

                        detail_response = requests.get(full_link)
                        print(full_link)
                        detail_soup = BeautifulSoup(detail_response.text, 'html.parser')
                        han_text = detail_soup.select_one('.han')
                        if han_text:  # .han 요소가 존재할 때만 처리
                            final_text = han_text.get_text().strip()
                            paragraphs.append(final_text.replace('\xa0', '').replace('\n', ''))
                            titles.append(title)
                        #for paragraph in paragraphs:
data={'title':title,'paragraph':paragraphs}
print(data)

  #revolution_main_table > tbody > tr:nth-child(7) > td:nth-child(2) > a > span
#https://gall.dcinside.com/mgallery/board/lists/?id=mvnogallery&page=5

In [ ]:
import pandas as pd
df = pd.DataFrame(data)

In [ ]:
len=[len(value) for key, value in data.items()]
len

In [ ]:
df.head()

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("searle-j/kote_for_easygoing_people")
model = AutoModelForSequenceClassification.from_pretrained("searle-j/kote_for_easygoing_people")

In [ ]:
print(model.config.id2label) #44개의 라벨링


In [ ]:
import torch
# 감정분석 함수 정의 (텍스트 레이블로 변환)
def analyze_sentiment_with_score(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1)
    max_prob, predicted_class = torch.max(probs, dim=1)
    predicted_class = predicted_class.item()
    max_prob = max_prob.item()

    # 모델 설정에 있는 id2label을 사용하여 숫자를 텍스트 레이블로 변환
    # id2label은 key가 int 또는 str일 수 있으므로 둘 다 확인
    if predicted_class in model.config.id2label:
        label = model.config.id2label[predicted_class]
    elif str(predicted_class) in model.config.id2label:
        label = model.config.id2label[str(predicted_class)]
    else:
        label = str(predicted_class)  # 매핑이 없는 경우 숫자를 그대로 반환

    return label, max_prob

# "title"과 "paragraph"를 결합하여 분석할 텍스트 생성
df["combined_text"] = df["title"] + " " + df["paragraph"]

# 각 행에 대해 감정분석 수행하여 텍스트 레이블 결과 컬럼 추가
df["sentiment_label"], df["sentiment_score"] = zip(*df["combined_text"].apply(analyze_sentiment_with_score))

# 결과 출력
print(df[["title", "sentiment_label"]])

In [ ]:
pd.set_option('display.max_colwidth', None)
df[["combined_text", "sentiment_label",'sentiment_score']].head()

In [ ]:
highest_comments = df.loc[df.groupby("sentiment_label")["sentiment_score"].idxmax()]
pd.reset_option('display.max_colwidth', None)


In [ ]:
 df['sentiment_label'].value_counts()

In [ ]:
# 결과 출력 (라벨, combined_text, sentiment_score 확인)
pd.set_option('display.max_colwidth', None)
highest_comments[["sentiment_label", "combined_text", "sentiment_score"]]

In [ ]:
n,m = list(map(int,input().split()))

s = []

def dfs():
    if len(s)==m:
        print(' '.join(map(str,s)))
        return

    for i in range(1,n+1):
        if i not in s:
            s.append(i)
            dfs()
            s.pop()

dfs()